# **AI and the Future of Work and Learning**

## Objectives

Extract --> Transform --> load 
- File 1: linkedin_job_postings.csv
- File 2: job_skills.csv
- File 3: ai_automation_risk_dataset.csv

Working with 3 files from 2 seperate datsets from kaggle:
1. 1.3M LinkedIn Jobs & Skills 2024 Dataset
2. AI Automation Risk by Job Role Dataset

Due to the large size of the original LinkedIn dataset, a subset of the data was selected and used for this project to ensure manageable processing.

Analyse --> Visulaise --> Hypothesis Testing 
- Hypothesis 1: Human-centred skills and automation risk
- Hypothesis 2: Hybrid digital and domain skills
- Hypothesis 3: Transferable skills and job resilience

Finally export cleaned, transformed data for PowerBI Dashboard and ML Model. 



---

## Extract --> Transform --> Load 

Import Libraries

In [ ]:
import pandas as pd #data processing 
import numpy as np #data processing
import re #cleaning text
import matplotlib.pyplot as plt #visualization
import seaborn as sns #visualization

Extract: Read & Load three CSV files into DataFrames

In [2]:
job_postings = pd.read_csv("../raw_data/linkedin_job_postings-trim.csv")
job_skills   = pd.read_csv("../raw_data/job_skills-trim.csv")
role_attrs   = pd.read_csv("../raw_data/ai_automation_risk_dataset-trim.csv")

Transform Data 

In [ ]:
def normalize_text(s):
    if pd.isna(s):
        return ""
    s = str(s).lower().strip()
    s = re.sub(r"[^a-z0-9\s]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

job_postings["job_link"] = job_postings["job_link"].astype(str).str.strip()
job_postings["job_title_norm"] = job_postings["job_title"].apply(normalize_text)

role_attrs["job_role"] = role_attrs["job_role"].astype(str)
role_attrs["job_role_norm"] = role_attrs["job_role"].apply(normalize_text)

#creates a cleaned version of job titles (job_title_norm)
#creates a cleaned version of job roles (job_role_norm)
#ensures job_link is consistently formatted


In [ ]:
#Mapping job_role from one file with job_title from 2nd csv file. 

role_norm_to_role = dict(
    zip(role_attrs["job_role_norm"], role_attrs["job_role"])
)

def map_role(title_norm):
    if title_norm in role_norm_to_role:
        return role_norm_to_role[title_norm]
    matches = [r for r in role_norm_to_role if r and r in title_norm]
    if matches:
        return role_norm_to_role[max(matches, key=len)]
    return np.nan

job_postings["job_role_mapped"] = job_postings["job_title_norm"].apply(map_role)

In [5]:
#Join job posting with role attributies to enriches each posting with the role-level attributes by joining on the mapped role.
df = job_postings.merge(
    role_attrs.add_prefix("role__"),
    left_on="job_role_mapped",
    right_on="role__job_role",
    how="left"
)

In [6]:
#Remove duplicates 
df_posting = (
    df
    .sort_values("job_link")
    .drop_duplicates("job_link")
    .reset_index(drop=True)
)

print("rows:", len(df_posting))
print("unique job_links:", df_posting["job_link"].nunique())
print("duplicate job_link rate:", df_posting["job_link"].duplicated().mean())

rows: 50015
unique job_links: 50015
duplicate job_link rate: 0.0


---

## Feature Engineering 

H1: Human-centred skills and automation risk

Job roles that require more human-centred skills (such as communication, creativity and problem-solving) have lower automation risk than roles based mainly on routine or repetitive tasks.